<a href="https://colab.research.google.com/github/diksha139/105StudentActivity/blob/main/Week8_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=02198ce0acba1d3b2c5740277a86cd1b34490155b80cbbe1083a29cab80cd0f1
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL DataFrame in JSON format") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
from google.colab import files
uploaded = files.upload()

Saving iot_devices.json to iot_devices.json


# Read the data into a Dataframe.

In [4]:
# spark is an existing SparkSession
df = spark.read.json("/content/iot_devices.json")
# Displays the content of the DataFrame to stdout
df.show()

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

In [5]:
df.printSchema()

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)



# Convert the Dataframe into a temporary view called iot.

In [6]:
df_new = df.createOrReplaceTempView("iot")

In [7]:
df_new = spark.sql("SELECT * FROM iot")
df_new.show()


+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

# Count how many devices are there from each country and display the output.

In [8]:
df.groupBy("cn").agg({'device_name':'count'}).orderBy("cn").show(10, False)

+-------------------+------------------+
|cn                 |count(device_name)|
+-------------------+------------------+
|                   |1810              |
|Afghanistan        |19                |
|Albania            |32                |
|Algeria            |34                |
|American Samoa     |3                 |
|Andorra            |4                 |
|Angola             |28                |
|Anguilla           |7                 |
|Antigua and Barbuda|60                |
|Argentina          |978               |
+-------------------+------------------+
only showing top 10 rows



In [9]:
spark.sql("SELECT cn, COUNT (device_name) AS no_of_device FROM iot GROUP BY cn ORDER BY cn").show(50)

+--------------------+------------+
|                  cn|no_of_device|
+--------------------+------------+
|                    |        1810|
|         Afghanistan|          19|
|             Albania|          32|
|             Algeria|          34|
|      American Samoa|           3|
|             Andorra|           4|
|              Angola|          28|
|            Anguilla|           7|
| Antigua and Barbuda|          60|
|           Argentina|         978|
|             Armenia|          34|
|               Aruba|           8|
|           Australia|        3119|
|             Austria|        1462|
|          Azerbaijan|          49|
|             Bahamas|          18|
|             Bahrain|          55|
|          Bangladesh|         153|
|            Barbados|          38|
|             Belarus|         123|
|             Belgium|         666|
|              Belize|          13|
|               Benin|           3|
|             Bermuda|          53|
|              Bhutan|      

# Display all the countries whose carbon dioxide level is more than 1400. Sort the output in descending order.

In [10]:

spark.sql("select cn, c02_level from iot where c02_level > 1400 order by c02_level desc").show()


+-------------+---------+
|           cn|c02_level|
+-------------+---------+
|     Bulgaria|     1599|
|       Poland|     1599|
|United States|     1599|
|        Spain|     1599|
|        Egypt|     1599|
|United States|     1599|
|       Taiwan|     1599|
|        Japan|     1599|
|      Germany|     1599|
|      Germany|     1599|
|      Germany|     1599|
|  Philippines|     1599|
|       Canada|     1599|
|       Canada|     1599|
|        Japan|     1599|
|       Canada|     1599|
|      Germany|     1599|
|United States|     1599|
|     Bulgaria|     1599|
|United States|     1599|
+-------------+---------+
only showing top 20 rows



# Select all countries' devices with high-levels of C02 and group by cca3 and order by device_ids (Hint: For high CO2 level, the LCD status will be RED).

In [11]:
spark.sql("select cn, count(distinct device_id) as device_id from iot where lcd == 'red' group by cn order by device_id").show()

+--------------------+---------+
|                  cn|device_id|
+--------------------+---------+
|British Indian Oc...|        1|
|         Ivory Coast|        1|
|          East Timor|        1|
|            Guernsey|        1|
|    Marshall Islands|        1|
|          Tajikistan|        1|
|             Andorra|        1|
|               Gabon|        1|
|             Réunion|        1|
|        Turkmenistan|        1|
|           Gibraltar|        1|
|Federated States ...|        1|
|             Liberia|        1|
|    Falkland Islands|        1|
|     Solomon Islands|        1|
|         Isle of Man|        2|
|                Fiji|        2|
|             Lesotho|        2|
|              Rwanda|        2|
|          San Marino|        2|
+--------------------+---------+
only showing top 20 rows



In [12]:
spark.sql("SELECT cn,device_name,c02_level,lcd FROM iot WHERE lcd == 'red' ").show()

+-----------------+--------------------+---------+---+
|               cn|         device_name|c02_level|lcd|
+-----------------+--------------------+---------+---+
|           Norway|   sensor-pad-2n2Pea|     1473|red|
|            Italy| device-mac-36TWSKiT|     1556|red|
|            Japan|sensor-pad-8xUD6p...|     1536|red|
|    United States|sensor-pad-10Bsyw...|     1470|red|
|            Italy|meter-gauge-11dlM...|     1544|red|
|    United States|sensor-pad-16aXmI...|     1425|red|
|    United States|meter-gauge-17zb8...|     1466|red|
|    United States|meter-gauge-19eg1...|     1531|red|
|            Japan|  sensor-pad-22oWV2D|     1522|red|
|           Canada|sensor-pad-24Pytz...|     1511|red|
|Republic of Korea|  device-mac-27P5wf2|     1597|red|
|Republic of Korea|sensor-pad-28Tsud...|     1502|red|
|          Ukraine|meter-gauge-47WsF9s8|     1454|red|
|           Sweden|meter-gauge-534fD...|     1446|red|
|    United States|sensor-pad-5410CW...|     1436|red|
|    Unite

# find out all devices in countries whose batteries need replacements.

In [13]:
spark.sql("select cn, count(distinct device_id) as device_id from iot where battery_level == 0 group by cn order by device_id desc limit 100").show()

+-----------------+---------+
|               cn|device_id|
+-----------------+---------+
|    United States|     6858|
|            China|     1415|
|Republic of Korea|     1217|
|            Japan|     1210|
|          Germany|      760|
|   United Kingdom|      650|
|           Canada|      612|
|           Russia|      600|
|           France|      582|
|           Brazil|      374|
|        Australia|      322|
|           Sweden|      293|
|            Italy|      287|
|           Poland|      278|
|      Netherlands|      251|
|            Spain|      223|
|           Taiwan|      207|
|            India|      189|
|                 |      177|
|          Ukraine|      149|
+-----------------+---------+
only showing top 20 rows

